In [1]:
#attempt to modify NTRU Encrypt by using R_p = F_p[S_N] rather than R_p = F_p[C_N] = F_p[x]/(x^N-1)

In [2]:
#the group of units of the symmetric group algebra are computed in Corollary 3.1, 3.2 of
#"Structure of the Unit Group of the Symmetric Group Algebra", Abhilash, Nandakumar

In [3]:
homomorphic_encryption = False

In [4]:
#define the parameters
N=3; p=5; q=67; d=2
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
assert q > (6*d+1)*p

In [5]:
#create the polynomial ring \Z[x]/(x^N-1)
R = SymmetricGroupAlgebra(ZZ,N); R

Symmetric group algebra of order 3 over Integer Ring

In [6]:
#a ternary polynomial in R = \Z[x]/(x^N-1) has d1 coeffs = +1, d2 coeffs = -1
def is_ternary(a,d1,d2):
    assert a in R
    coeffs = [pair[1] for pair in list(a)]
    return list(coeffs).count(+1) == d1 and list(coeffs).count(-1) == d2

In [7]:
#create the symmetric group algebra F_q[S_N]
R_q = SymmetricGroupAlgebra(GF(q),N); R_q

Symmetric group algebra of order 3 over Finite Field of size 67

In [8]:
#create the quotient polynomial ring F_p[S_N]
R_p = SymmetricGroupAlgebra(GF(p),N); R_p

Symmetric group algebra of order 3 over Finite Field of size 5

In [9]:
#create a random unit of the symmetric group algebra over a finite field
#essentially if K_q[S_N] \cong M_{n_1}(K_{q^{m_1}}) \oplus ... \oplus M_{n_r}(K_{q^{m_r}}) then
#\mathcal{U}(K_q[S_N]) \cong GL_{n_1}(K_{q^{m_1}}) \oplus ... \oplus GL_{n_r}(K_{q^{m_r}})
#and the number n_k, m_k can be read off from the Brattelli diagram.
#however, only ~1/p elements are not invertible by determinant considerations, so most elements should be invertible
def random_unit_SGA(p,n,max_iter=1,ternary=False,d1=0,d2=0):
    assert p.is_prime()
    for i in range(max_iter):
        if ternary:
            random_element = rand_ternary(N,d1,d2)
        else:
            random_element = R.random_element()
        try:
            R_p(random_element).inverse()
            return random_element
        except ValueError:
            continue
    print("Could not find invertible element of symmetric group algebra.")

In [10]:
#choose a random element of \mathcal{T}(d_1,d_2)
def rand_ternary(N,d1,d2):
    G = R.group()
    group_order = G.cardinality()
    coeffs = (group_order-(d1+d2))*[0] + d1*[+1] + d2*[-1]
    sigma = Permutations(group_order).random_element()
    r = sum([coeffs[sigma[i]-1]*R(G[i]) for i in range(group_order)])
    assert is_ternary(r,d1,d2)
    return r

In [11]:
#create an element f of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
f = random_unit_SGA(p,N,max_iter=100,ternary=True,d1=d+1,d2=d)
print(f)

[1, 3, 2] - [2, 1, 3] - [2, 3, 1] + [3, 1, 2] + [3, 2, 1]


In [12]:
#create an element g of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
g = rand_ternary(N,d,d)
print(g)

[1, 2, 3] - [1, 3, 2] + [2, 1, 3] - [2, 3, 1]


In [13]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f).inverse(); print(F_p)
assert F_p * R_p(f) == 1

[1, 3, 2] + 4*[2, 1, 3] + 4*[2, 3, 1] + [3, 1, 2] + [3, 2, 1]


In [14]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f).inverse(); print(F_q)
assert F_q * R_q(f) == 1

[1, 3, 2] + 66*[2, 1, 3] + 66*[2, 3, 1] + [3, 1, 2] + [3, 2, 1]


In [15]:
#define the private key used to decrypt messages
def private_key(f):
    return (f,R_q(f).inverse())

In [16]:
#define the public key using f(x) (mod p), g(x) (mod p)
def public_key(f,g,q):
    F_q = R_q(f).inverse()
    return F_q * R_q(g)

In [17]:
print(public_key(f,g,q))

64*[1, 2, 3] + 66*[1, 3, 2] + 64*[2, 1, 3] + 66*[2, 3, 1] + 4*[3, 1, 2] + 4*[3, 2, 1]


In [18]:
#define a plaintext message m(x) with coefficients satisfying -p/2 < m_i <= p/2
#i.e. m is the center lift of a polynomial in R_p
def plaintext(m,p,N):
    assert all([-p/2 < m[i] <= +p/2 for i in range(N)])
    G = R.group()
    return sum(m[i]*R(G[i]) for i in range(G.cardinality()))

In [19]:
#compute the ciphertext e(x)
def ciphertext(m,f,g,r,p,q,N,d):
    h = public_key(f,g,q)
    m = plaintext(m,p,N)
    e = R_q(p*h*r + m)
    return e

In [20]:
#begin decryption process
def a(f,e):
    return R_q(f*e)

In [37]:
#define the center lift of a from R_q to an element of R = \Z[x]/(x^N-1)
#map coefficients from F_q to Z/qZ (the additive group) and center lift them to \Z
def center_lift(a,modulus):
    G = R.group()
    coeffs = [a(f,e)[g] for g in G]
    print(coeffs)
    center_lift_coeffs = [Zmod(modulus)(coeff).lift_centered() for coeff in coeffs]
    print(center_lift_coeffs)
    return sum(center_lift_coeffs[i]*R(G[i]) for i in range(G.cardinality()))

In [22]:
#compute b(x) = F_p(x) * a(x) (mod p) by first center lifting a(x) to R
def b(f,a,q):
    F_p = R_p(f).inverse()
    return R_p(F_p * center_lift(a,q))

In [23]:
#define the final center_lift for decryption where e = ciphertext, f = secret key, p = prime, q = modulus
def decrypt(e,f,p,q):
    return center_lift(b(f,a(f,e),q),p)

In [24]:
#EXAMPLE

In [25]:
m = plaintext([1,-1,1,1,0,-1],p,N); m

[1, 2, 3] - [1, 3, 2] + [2, 1, 3] + [2, 3, 1] - [3, 2, 1]

In [26]:
r = rand_ternary(N,d,d); r

[1, 3, 2] - [2, 1, 3] + [3, 1, 2] - [3, 2, 1]

In [27]:
e = ciphertext(m,f,g,r,p,q,N,d); e

52*[1, 2, 3] + 22*[1, 3, 2] + 60*[2, 1, 3] + 60*[2, 3, 1] + 22*[3, 1, 2] + 52*[3, 2, 1]

In [28]:
a(f,e)

52*[1, 2, 3] + 15*[1, 3, 2] + 15*[3, 1, 2] + 52*[3, 2, 1]

In [38]:
center_lift(a(f,e),q)

TypeError: 'SymmetricGroupAlgebra_n_with_category.element_class' object is not callable

In [ ]:
b(f,a(f,e),q)

In [ ]:
decrypt(e,f,p,q)